In [ ]:
from pydicom import dcmread

In [ ]:
from pathlib import Path

In [ ]:
src_folder = Path('dataset-to-commit/dataset-duplicate')


In [ ]:
labels = [label.name for label in src_folder.iterdir() if label.is_dir()]

In [ ]:
labels

In [ ]:
tgt_folder = Path('transformed_dataset')

In [ ]:
import numpy as np
def dcm_loader(path):
    dcm = dcmread(path)
    arr = dcm.pixel_array
    height, width = arr.shape
    if width < height:
        pad_width = height - width
        arr = np.pad(arr, ((0, 0), (0, pad_width)), mode='constant', constant_values=0)
    elif width > height:
        count = width // height
        rest = width % height
        # Repeat vertically
        arr_repeated = np.tile(arr, (count, 1))
        # Pad the rest with zeros at the bottom
        if rest > 0:
            arr_pad = np.zeros((rest, width), dtype=arr.dtype)
            arr_repeated = np.vstack([arr_repeated, arr_pad])
        arr = arr_repeated
    # else: already square
    arr = arr / arr.max() * 255.  # Normalize to 0-255
    arr = arr.astype(np.uint8)  # Convert to uint8
    img = Image.fromarray(arr)
    return img

In [ ]:
import numpy as np
from PIL import Image
def dcm_loader(path):
    img_array = []
    imgs = []
    dcm = dcmread(path)
    arr = dcm.pixel_array
    arr = arr / arr.max() * 255
    arr = arr.astype(np.uint8)
    height, width = arr.shape
    if height > width:
        arr = arr.transpose()
    height, width = arr.shape
    proportion = int(np.ceil(width / height))
    arr = np.pad(arr, ((0, 0), (0, int(height * proportion - width))), mode='constant', constant_values=0)
    for i in range(proportion):
        img_array.append(arr[:, height*i:height*(i+1)])
    for img in img_array:
        imgs.append(Image.fromarray(img))
    return imgs
    

In [ ]:
src_folder = Path('official_data/train_ds')

In [ ]:
labels = [folder.name for folder in src_folder.iterdir()]

In [ ]:
labels

In [ ]:
target_folder = Path('splitted_imgs')
for label in labels:
    sub_folder = src_folder / label
    for file in sub_folder.rglob("**/*.dcm"):
        imgs = dcm_loader(file)
        for index, img in enumerate(imgs):
            tgt_file = target_folder / label / f"{file.stem}_{index}.png"
            tgt_file.parent.mkdir(parents=True, exist_ok=True)
            img.save(tgt_file)
    print(f'{label}: Done')

In [ ]:
src_folder = Path('official_data/val_ds')

In [ ]:
for label in labels:
    sub_folder = src_folder / label
    for file in sub_folder.rglob("**/*.dcm"):
        imgs = dcm_loader(file)
        for index, img in enumerate(imgs):
            tgt_file = target_folder / label / f"{file.stem}_{index}.png"
            tgt_file.parent.mkdir(parents=True, exist_ok=True)
            img.save(tgt_file)
    print(f'{label}: Done')

In [ ]:
src_folder = Path('official_data/test_ds')

In [ ]:
for label in labels:
    sub_folder = src_folder / label
    for file in sub_folder.rglob("**/*.dcm"):
        imgs = dcm_loader(file)
        for index, img in enumerate(imgs):
            tgt_file = target_folder / label / f"{file.stem}_{index}.png"
            tgt_file.parent.mkdir(parents=True, exist_ok=True)
            img.save(tgt_file)
    print(f'{label}: Done')